In [1]:
import time
import os

import mysql.connector
from mysql.connector import Error
from faker import Faker
Faker.seed(33422)

fake = Faker()

In [5]:
db_user="root"
db_pass="root"
db_host="127.0.0.1"
#port=3306
db_name="checkapp"

In [135]:
try:
    conn = mysql.connector.connect(host=db_host, database=db_name,
                                   user=db_user, password=db_pass)

    if conn.is_connected():
        cursor = conn.cursor()
        row = generate_user_info()
        query = """INSERT INTO `user` \
        (company_id, name, rut, role, email, password, shift_id, picture) \
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"""
        cursor.execute(query, row)

        conn.commit()
except Error as e :
    print ("error", e)
    pass
except Exception as e:
    print ("Unknown error %s", e)
finally:
    #closing database connection.
    if(conn and conn.is_connected()):
        conn.commit()
        cursor.close()
        conn.close()

In [8]:
try:
    conn = mysql.connector.connect(host=db_host, database=db_name,
                                   user=db_user, password=db_pass)
    if conn.is_connected():
        cursor = conn.cursor()
        q = "SHOW TABLES;"
        cursor.execute(q)
        for table in cursor:
            print(table)

except Error as e :
    print ("error", e)
    pass
except Exception as e:
    print ("Unknown error %s", e)
finally:
    #closing database connection.
    if(conn and conn.is_connected()):
        conn.commit()
        cursor.close()
        conn.close()

('attendance',)
('company',)
('device',)
('qr',)
('shift',)
('user',)


In [ ]:
'INSERT INTO `user` (company_id, name, rut, role, email, password, shift_id, picture) \
VALUES ("%s", "%s", "%s", %s, "%s", "%s", "%s", "%s");'

In [98]:
import requests
import random

genders = ["men", "women"]
digito_verificador = list(range(1, 10)) + ['K']
ids = set()
def get_random_img_bytes(gender):
    _id = random.randint(1, 75)
    while (gender, _id) in ids:
        _id = random.randint(1, 75)
    ids.add((gender, _id))
    url = f"https://randomuser.me/api/portraits/{gender}/{_id}.jpg"    
    response = requests.get(url, stream=True)
    return response.content

In [1]:
import hashlib
  
def get_password_from_email(email):
    passw = email.split('@')[0]
    return hashlib.md5(passw.encode()).hexdigest()

In [2]:
hashlib.md5("cristo".encode()).hexdigest()

'71e099e35bbc5c5f64abe0c0dda9091d'

In [125]:
def generate_user_info():
    gender = random.sample(genders, 1)[0]
    if gender == 'men':
        name = fake.first_name_male()

    else:
        name = fake.first_name_female()
    name = name + " " + fake.last_name()
    picture = get_random_img_bytes(gender) 
    company = random.randint(1, 2)
    shift = random.randint(1, 2)
    last_name = fake.last_name()
    email = fake.email()
    password = get_password_from_email(email)
    role = "cringe"
    rut = f"{random.randint(10 * 10**6, 20 * 10**6)}-{random.sample(digito_verificador, 1)[0]}"
    return (company, name, rut, role, email, password, shift, picture)

In [154]:
try:
    conn = mysql.connector.connect(host=db_host, database=db_name,
                                   user=db_user, password=db_pass)
    if conn.is_connected():
        cursor = conn.cursor()
        _id = 18
        q1 = f"SELECT u.id, u.company_id, u.shift_id FROM user u WHERE u.id = {_id};"
        cursor.execute(q)
        for (user_id, company_id, shift_id) in cursor:
            pass
        print(user_id)        

except Error as e :
    print ("error", e)
    pass
except Exception as e:
    print ("Unknown error %s", e)
finally:
    #closing database connection.
    if(conn and conn.is_connected()):
        conn.commit()
        cursor.close()
        conn.close()

18


In [223]:
from random import randrange
from datetime import timedelta, datetime

WEEKDAYS = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
shifts_days = {
    1 : ['monday', 'tuesday', 'wednesday', 'thursday', 'friday'],
    2 : ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
         }
shifts_time = {
    1 : ["09:00:00", "17:30:00"],
    2 : ["06:00:00", "19:00:00"],
}

def random_date(start):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
#     delta = end - start
#     int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    int_delta = 7000
    random_second = randrange(-int_delta//2, int_delta//2)
    return start + timedelta(seconds=random_second)

def get_attendance_dtime(user_id, shift, day, month):
    arrival, leave = shifts_time[shift]
    d1 = datetime.strptime(f'2022-{month}-{day} {arrival}', '%Y-%m-%d %H:%M:%S')
    if WEEKDAYS[d1.weekday()] not in shifts_days[shift]:
        return None
    r1 = random_date(d1)
    d2 = datetime.strptime(f'2022-{month}-{day} {leave}', '%Y-%m-%d %H:%M:%S')
    r2 = random_date(d2)
    if (r1 - d1).total_seconds()//60 > 30:
        comment_arr = "LATE_ARRIVAL"
    else:
        comment_arr = "ON_TIME"
    if (d2 - r2).total_seconds()//60 > 30:
        comment_leav = "EARLY_LEAVE"
    else:
        comment_leav = "ON_TIME"
    return [(user_id, "-40,70", str(r1), "CHECK_IN", False, comment_arr, arrival),
           (user_id, "-40,70", str(r2), "CHECK_OUT", False, comment_leav, leave)]
            

In [224]:
for i in range(1, 30):
    print(get_attendance_dtime(0, 2, i, 4))

[(0, '-40,70', '2022-04-01 05:28:40', 'CHECK_IN', False, 'ON_TIME', '06:00:00'), (0, '-40,70', '2022-04-01 18:06:30', 'CHECK_OUT', False, 'EARLY_LEAVE', '19:00:00')]
[(0, '-40,70', '2022-04-02 06:54:19', 'CHECK_IN', False, 'LATE_ARRIVAL', '06:00:00'), (0, '-40,70', '2022-04-02 19:32:24', 'CHECK_OUT', False, 'ON_TIME', '19:00:00')]
[(0, '-40,70', '2022-04-03 05:59:31', 'CHECK_IN', False, 'ON_TIME', '06:00:00'), (0, '-40,70', '2022-04-03 19:15:20', 'CHECK_OUT', False, 'ON_TIME', '19:00:00')]
[(0, '-40,70', '2022-04-04 05:48:49', 'CHECK_IN', False, 'ON_TIME', '06:00:00'), (0, '-40,70', '2022-04-04 19:21:22', 'CHECK_OUT', False, 'ON_TIME', '19:00:00')]
[(0, '-40,70', '2022-04-05 05:34:42', 'CHECK_IN', False, 'ON_TIME', '06:00:00'), (0, '-40,70', '2022-04-05 18:14:35', 'CHECK_OUT', False, 'EARLY_LEAVE', '19:00:00')]
[(0, '-40,70', '2022-04-06 06:46:47', 'CHECK_IN', False, 'LATE_ARRIVAL', '06:00:00'), (0, '-40,70', '2022-04-06 18:02:09', 'CHECK_OUT', False, 'EARLY_LEAVE', '19:00:00')]
[(0, '

In [225]:
qatt = """INSERT INTO attendance \
    (user_id, location, event_time, event_type, pending, comments, expected_time) \
    VALUES (%s, %s, %s, %s, %s, %s, %s);"""

In [229]:
try:
    conn = mysql.connector.connect(host=db_host, database=db_name,
                                   user=db_user, password=db_pass)
    if conn.is_connected():
        cursor = conn.cursor()
        for _id in range(18, 28):
            q1 = f"SELECT u.id, u.company_id, u.shift_id FROM user u WHERE u.id = {_id};"
            cursor.execute(q1)
            for (user_id, company_id, shift_id) in cursor:
                pass
            print(user_id)
            for day in range(1, 5):
                atts = get_attendance_dtime(user_id, shift_id, day, 5)
                if atts:
                    for att in atts:
                        cursor.execute(qatt, att)

except Error as e :
    print ("error", e)
    pass
except Exception as e:
    print ("Unknown error %s", e)
finally:
    #closing database connection.
    if(conn and conn.is_connected()):
        conn.commit()
        cursor.close()
        conn.close()

18
19
20
21
22
23
24
25
26
27
